#### 한국어 코퍼스 전처리 (*Torchtext 없이 해보기)
- 데이터셋 : Korpora에서 로딩
- 형태소분석기 설정
- 단어사전 생성

[1] 모듈 로딩 및 데이터 준비 <HR>

In [1]:
## 모듈 로딩
from Korpora import Korpora                         # Open Korean Dataset
from konlpy.tag import *                            # 형태소 분석기
import spacy                                        # 형태소 분석기

from nltk.tokenize import word_tokenize, sent_tokenize, wordpunct_tokenize
from torch.utils.data import Dataset, DataLoader    # Pytorch Dataset 관련 모듈
from nltk.corpus import stopwords
import pandas as pd
import numpy as np

In [2]:
### 데이터 로딩
nsmc = Korpora.load('nsmc')


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\KDP-50\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\KD

In [3]:
# nsmc.train[0], nsmc.test[0]

In [3]:
# Test 데이터셋을 DataFrame으로 로딩
nsmcDF = pd.DataFrame(nsmc.test)
nsmcDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


[3] 텍스트 데이터 전처리 <hr>
- 토큰화 / 정제 (불용어, 구두점, 띄어쓰기, 오타 등등 처리)
- 단어사전 생성
- 문장 ==> 수치화

In [5]:
### 형태소 분석기
# # 한나눔
# han = Hannanum()

# # 꼬꼬마
# kkma = Kkma()

# # OKT
# okt = Okt()

In [6]:
sample = nsmc.test.texts[:100]

In [ ]:
for i in sample[:10]:
    print(i)

- 불용어 & 구두점 제거

In [8]:
import re
import string
punc = string.punctuation

In [12]:
all_text=[]
for text in sample[::-1]:
    text=text.replace('\n','')
    text=text.replace(punc,'')
    if len(text): all_text.append(text)

In [ ]:
for tokenList in all_text[:10]:
    print(tokenList, end='\n')

In [34]:
# makestr=",".join(nsmc.test.texts)
makestr=",".join(all_text)

In [35]:
# 불용어,구두점 제거 (정규식) 
import re
import string

# newmake = re.sub('[\,.? ]', '', makestr)

In [36]:
# 형태소 분석

# han_result=han.morphs(makestr)
# kkma_result=kkma.morphs(makestr)
okt_result=okt.morphs(makestr)
# okt_result=okt.morphs(newmake)

In [37]:
all_tokens=[]
for text in okt_result:
    all_tokens.append( wordpunct_tokenize(text) )

In [ ]:
len(all_tokens)

In [39]:
# 한국어 불용어 제거
def remove_stopwords(tokens):
    return [token for token in tokens if token not in stopwords]

In [40]:
STOP_PATH = '../Data/stopwords-ko.txt'

In [41]:
with open(STOP_PATH, 'r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()

In [42]:
mytoken = remove_stopwords(all_tokens)

In [43]:
for tokenList in mytoken:
    for token in tokenList:
        if (token in punc) & (token == 'ㅋ'):
            tokenList.remove(token)

In [ ]:
for tokenList in mytoken:
    print(tokenList)

- 단어별 빈도수 계산

In [45]:
# 토큰을 키로 해서 빈도수 저장
token_freg={} # dict에서는 key값을 찾아봄

# 라인(줄)별 토큰을 읽어서 빈도 체크
for tokenList in mytoken:
    for token in tokenList:
        # 카운트시, 토큰 key가 존재하지 않으면 key 추가
        if token not in token_freg:
            token_freg[token] = 1
        # 카운트시, 이미 존재하는 토큰 key는 1 증가
        else:
            token_freg[token] += 1

In [ ]:
token_freg

In [ ]:
# 단어들 별로 빈도수 저장
freqsDict={}
for k, v in token_freg.items():
    if v not in freqsDict:
        freqsDict[v]=[k]
    else:
        freqsDict[v].append(k)
print(freqsDict)

In [ ]:
# 단어등장 횟수, 단어 개수 및 단어이름 별로 빈도수 저장
freqsDict={}
for k, v in token_freg.items():
    if v not in freqsDict:
        # freqsDict[v]=[k]
        # freqsDict[v]=1
        freqsDict[v]=[1, [k]]
    else:
        # freqsDict[v].append(k)
        # freqsDict[v]+=1
        freqsDict[v][0]+=1
        freqsDict[v][1].append(k)
print(freqsDict)

In [ ]:
# [(단어나온 횟수, [단어개수, [단어이름]]) ...]
sorted(freqsDict.items(), reverse=True)

In [50]:
## 빈도가 높게 나오는 순서대로 단어 정렬
#                k, v -----------------------------|
storedTokens=sorted(token_freg.items(), reverse=True, key=lambda x:x[1])

In [51]:
## 단어사전 생성 및 초기화
## 특수토큰 : 'PAD', 'OOV' 또는 'UNK'
PAD_TOKEN, OOV_TOKEN='PAD', 'OOV'
vocab={PAD_TOKEN:0, OOV_TOKEN:1}

In [ ]:
## 데이터 코퍼스에서 추출된 토큰(단어)들
# vocab[]

for idx, tk in enumerate(storedTokens, 2):
    vocab[tk[0]] = idx
print(vocab)

In [ ]:
## 텍스트 문장 ===> 수치화 [인코딩]
encodingData=[]
for tokenList in all_tokens:
    sent=[]
    print(f'문장: {tokenList}')
    for token in tokenList:
        sent.append(vocab[token])
    
    # 인코딩 된 문장 저장
    encodingData.append(sent)
    print(f'==>인코딩: {sent}\n')